Goal of this case study is to predict survivors based on Titanic dataset and compare different alghoritms with different hiperparameters. At the end we can choose the best one for this task.

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/home/dawid/Dokumenty/Data Science Pro/Zjazd 9/titanic.csv', decimal=',')
data.head() 

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,home.dest
0,3,0,"Abbing, Mr. Anthony",male,42.0,0,0,C.A. 5547,7.55,NaN,S,NaN
1,3,0,"Abbott, Master. Eugene Joseph",male,13.0,0,2,C.A. 2673,20.25,NaN,S,"East Providence, RI"
2,3,0,"Abbott, Mr. Rossmore Edward",male,16.0,1,1,C.A. 2673,20.25,NaN,S,"East Providence, RI"
3,3,1,"Abbott, Mrs. Stanton (Rosa Hunt)",female,35.0,1,1,C.A. 2673,20.25,NaN,S,"East Providence, RI"
4,3,1,"Abelseth, Miss. Karen Marie",female,16.0,0,0,348125,7.65,NaN,S,"Norway Los Angeles, CA"


1. Data preparation

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1046 entries, 0 to 1045
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1046 non-null   int64  
 1   survived   1046 non-null   int64  
 2   name       1046 non-null   object 
 3   sex        1046 non-null   object 
 4   age        1046 non-null   float64
 5   sibsp      1046 non-null   int64  
 6   parch      1046 non-null   int64  
 7   ticket     1046 non-null   object 
 8   fare       1045 non-null   float64
 9   cabin      272 non-null    object 
 10  embarked   1044 non-null   object 
 11  home.dest  685 non-null    object 
dtypes: float64(2), int64(4), object(6)
memory usage: 98.2+ KB


In [4]:
y = data['survived'] #it's our variable that we'd like to predict

In [5]:
data['ticket'].unique() #it's something like ID of ticket, we can't reasonably 
#use this information

array(['C.A. 5547', 'C.A. 2673', '348125', '348122', 'P/PP 3381',
       'SOTON/O2 3101284', '2657', 'C 7076', '341826', '7546', '392091',
       '2699', '248744', '3474', 'SOTON/O2 3101287', 'SOTON/O.Q. 3101311',
       'SOTON/O.Q. 3101312', '24160', '373450', '113781', '2223',
       '350046', 'C 4001', '19952', '347082', '3101281', '347091',
       '350043', '347075', '347466', '231945', 'C.A. 34050', '13502',
       '112050', '349202', '226875', '11769', '349237', '349911',
       'PC 17609', '244346', 'SOTON/O.Q. 3101310', '347077', '350054',
       '2696', '2692', 'SOTON/O.Q. 3101309', 'PC 17757', '2627', '2694',
       'PC 17477', '347468', '2687', '2679', '3101278', '2666',
       'A/4 31416', '2623', '29108', 'C.A. 31030', '349248', '28551',
       'C.A./SOTON 34068', '2663', '2691', '19877', '27042', '330844',
       '324669', 'S.O.P. 1166', 'PC 17558', '11813', '2908', '13050',
       '244358', '323951', '230136', '11751', '248698', '111369',
       '347068', '28404', 'PP 43

In [6]:
data.drop(columns='ticket', axis=1, inplace=True)

In [7]:
data.drop(columns='fare', axis=1, inplace=True)
#First thing we can see is that fare - ticket price is object Dtype and should be float. 
#But I decided not to use this variable at all, beacouse: 1) we can't be sure about currency and
#correctness (probably it's the summarized price for all people from the ticket) 
#AND this variable is higly correlated with cabin and pclass. 

In [8]:
#There is big amount of Nan's BUT it could be very useful information so we'll try to do sth
#We'll only use the letter sign - we assume that it's a deck designation. 
data.cabin.str[0].value_counts()

C    86
B    63
D    42
E    38
A    19
F    18
G     5
T     1
Name: cabin, dtype: int64

In [9]:
data["deck"] = data.cabin.str[0].fillna("Unknown").replace({"G":"F", "T":"F"})
#There is very little
#samples in classes T and G so we can add them to class F. Nan's we turn into class Unknown.

In [11]:
data.drop(['cabin'], axis=1, inplace=True) #now we can delete column cabin

In [12]:
data['embarked'].fillna(data['embarked'].value_counts().index[0], inplace=True)
#Column 'embarked' it's the port from which passengers departed. It seems as usefull information
#but it's probably highly correlated with cabin (now deck) and it's probably more informative but 
#on the other hand it can be more informative for us than ID's of cabins that we've alreade 
#established it's hard to locate on ship
#We'll use this column and fill Nan's with most common value which is 'S'. There is only 2 Nan's
#so it shouldn't be a mistake.

In [13]:
#home.dest (home destination) is the column that 
#we will not use as it does not carry any information related to the disaster

In [14]:
X = pd.get_dummies(data [['pclass', 'sex', 'age', 'sibsp', 'parch', 
                          'embarked', 'deck']], drop_first=True)
X.head() #'sex','embarked' and 'deck' are categorical columns so they should be encoded.

,pclass,age,sibsp,parch,sex_male,embarked_Q,embarked_S,deck_B,deck_C,deck_D,deck_E,deck_F,deck_Unknown
0,3,42.0,0,0,1,0,1,0,0,0,0,0,1
1,3,13.0,0,2,1,0,1,0,0,0,0,0,1
2,3,16.0,1,1,1,0,1,0,0,0,0,0,1
3,3,35.0,1,1,0,0,1,0,0,0,0,0,1
4,3,16.0,0,0,0,0,1,0,0,0,0,0,1


2. Modeling

2.1 Decision Tree Classifier

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
#now we split data into training and testing. We can assume basing on dataset size that 80:20 
#should be correct ratio

In [56]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold

In [25]:
model_dt = DecisionTreeClassifier()
model_dt.fit(X_train, y_train)
y_pred_dt = model_dt.predict(X_test)
accuracy_score(y_test, y_pred_dt)
#~70% accuracy when we fit Decision Tree Classifier without parameter optimization

0.719047619047619

In [41]:
np.arange(1, 21)

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20])

In [52]:
kfold = StratifiedKFold(10, shuffle=True, random_state=123)

model_dt = DecisionTreeClassifier(min_samples_leaf=10)
params = {"min_samples_leaf": np.arange(1, 21),
          'max_depth': [np.arange(1, 16), None],
         'criterion': ['gini', 'entropy']}
optimizer_dt = GridSearchCV(model_dt, 
                         params, 
                         scoring="accuracy", 
                         cv=kfold, 
                         n_jobs=-1)
optimizer_dt.fit(X_train, y_train)

y_pred_dt = optimizer_dt.best_estimator_.predict(X_test)
print(optimizer_dt.best_params_)
print(accuracy_score(y_test, y_pred_dt));

{'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 5}
0.7904761904761904


/home/dawid/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
400 fits failed out of a total of 800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
400 fits failed with the following error:
Traceback (most recent call last):
  File "/home/dawid/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/dawid/anaconda3/lib/python3.9/site-packages/sklearn/tree/_classes.py", line 937, in fit
    super().fit(
  File "/home/dawid/anaconda3/lib/python3.9/site-packages/sklearn/tree/_classes.py", line 305, in fit
    if max_depth <= 0:
ValueError: The truth value of an array with more

In [55]:
model_dt = DecisionTreeClassifier(criterion='gini', max_depth=None, min_samples_leaf=5)
model_dt.fit(X_train, y_train)
y_pred_dt = model_dt.predict(X_test)
accuracy_score(y_test, y_pred_dt)
#Decision Tree Classifier with optimal parameters. Accuracy increases to ~79%

0.7904761904761904

2.2 Logistic regression

In [60]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [61]:
model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)
y_pred_lr = model_lr.predict(X_test)
accuracy_score(y_test, y_pred_lr)
#~78% accuracy when we use Linear Regression model without optimization

/home/dawid/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7857142857142857

In [68]:
#Optimization
kfold = StratifiedKFold(10, shuffle=True, random_state=123)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(solver='saga'))
])
params = [
    {
        'model__penalty': ['none']
    },
    {
        'model__penalty': ['l1', 'l2'],
        'model__C': [0.01, 0.1, 1, 10, 100, 1000]
    }
]

optimizer_lr = GridSearchCV(pipeline,
                           params,
                           scoring='accuracy',
                           cv=kfold,
                           n_jobs=-1)
optimizer_lr.fit(X_train, y_train)
optimizer_lr.best_params_

{'model__C': 0.1, 'model__penalty': 'l2'}

In [70]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(penalty='l2', C=0.1))
])
pipeline.fit(X_train, y_train)
y_pred_lr = pipeline.predict(X_test)
accuracy_score(y_test, y_pred_lr)
#after optimization we are at the same level of accuracy

0.7714285714285715